In [4]:
# ------------------------------------------------------------------------
# Copyright (c) 2022 megvii-model. All Rights Reserved.
# ------------------------------------------------------------------------

'''
Simple Baselines for Image Restoration

@article{chen2022simple,
  title={Simple Baselines for Image Restoration},
  author={Chen, Liangyu and Chu, Xiaojie and Zhang, Xiangyu and Sun, Jian},
  journal={arXiv preprint arXiv:2204.04676},
  year={2022}
}
'''
import sys 
sys.path.append('/home/lllei/AI_localization/NAFNet')

import torch
import torch.nn as nn
import torch.nn.functional as F
from basicsr.models.archs.arch_util import LayerNorm2d
from basicsr.models.archs.local_arch import Local_Base
from torchsummary import summary


class SimpleGate(nn.Module):
    def forward(self, x):
        x1, x2 = x.chunk(2, dim=1)
        return x1 * x2

class NAFBlock(nn.Module):
    def __init__(self, c, DW_Expand=2, FFN_Expand=2, drop_out_rate=0.):
        super().__init__()
        dw_channel = c * DW_Expand
        self.conv1 = nn.Conv2d(in_channels=c, out_channels=dw_channel, kernel_size=1, padding=0, stride=1, groups=1, bias=True)
        self.conv2 = nn.Conv2d(in_channels=dw_channel, out_channels=dw_channel, kernel_size=3, padding=1, stride=1, groups=dw_channel,
                               bias=True)
        self.conv3 = nn.Conv2d(in_channels=dw_channel // 2, out_channels=c, kernel_size=1, padding=0, stride=1, groups=1, bias=True)
        
        # Simplified Channel Attention
        self.sca = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels=dw_channel // 2, out_channels=dw_channel // 2, kernel_size=1, padding=0, stride=1,
                      groups=1, bias=True),
        )

        # SimpleGate
        self.sg = SimpleGate()

        ffn_channel = FFN_Expand * c
        self.conv4 = nn.Conv2d(in_channels=c, out_channels=ffn_channel, kernel_size=1, padding=0, stride=1, groups=1, bias=True)
        self.conv5 = nn.Conv2d(in_channels=ffn_channel // 2, out_channels=c, kernel_size=1, padding=0, stride=1, groups=1, bias=True)

        self.norm1 = LayerNorm2d(c)
        self.norm2 = LayerNorm2d(c)

        self.dropout1 = nn.Dropout(drop_out_rate) if drop_out_rate > 0. else nn.Identity()
        self.dropout2 = nn.Dropout(drop_out_rate) if drop_out_rate > 0. else nn.Identity()

        self.beta = nn.Parameter(torch.zeros((1, c, 1, 1)), requires_grad=True)
        self.gamma = nn.Parameter(torch.zeros((1, c, 1, 1)), requires_grad=True)

    def forward(self, inp):
        x = inp

        x = self.norm1(x)

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.sg(x)
        x = x * self.sca(x)
        x = self.conv3(x)

        x = self.dropout1(x)

        y = inp + x * self.beta

        x = self.conv4(self.norm2(y))
        x = self.sg(x)
        x = self.conv5(x)

        x = self.dropout2(x)

        return y + x * self.gamma


class NAFNet(nn.Module):

    def __init__(self, img_channel=3, width=16, middle_blk_num=1, enc_blk_nums=[], dec_blk_nums=[]):
        super().__init__()

        self.intro = nn.Conv2d(in_channels=img_channel, out_channels=width, kernel_size=3, padding=1, stride=1, groups=1,
                              bias=True)
        self.ending = nn.Conv2d(in_channels=width, out_channels=img_channel, kernel_size=3, padding=1, stride=1, groups=1,
                              bias=True)

        self.encoders = nn.ModuleList()
        self.decoders = nn.ModuleList()
        self.middle_blks = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()

        chan = width
        for num in enc_blk_nums:
            self.encoders.append(
                nn.Sequential(
                    *[NAFBlock(chan) for _ in range(num)]
                )
            )
            self.downs.append(
                nn.Conv2d(chan, 2*chan, 2, 2)
            )
            chan = chan * 2

        self.middle_blks = \
            nn.Sequential(
                *[NAFBlock(chan) for _ in range(middle_blk_num)]
            )

        for num in dec_blk_nums:
            self.ups.append(
                nn.Sequential(
                    nn.Conv2d(chan, chan * 2, 1, bias=False),
                    nn.PixelShuffle(2)
                )
            )
            chan = chan // 2
            self.decoders.append(
                nn.Sequential(
                    *[NAFBlock(chan) for _ in range(num)]
                )
            )

        self.padder_size = 2 ** len(self.encoders)

    def forward(self, inp):
        B, C, H, W = inp.shape
        inp = self.check_image_size(inp)

        x = self.intro(inp)

        encs = []

        for encoder, down in zip(self.encoders, self.downs):
            x = encoder(x)
            encs.append(x)
            x = down(x)

        x = self.middle_blks(x)

        for decoder, up, enc_skip in zip(self.decoders, self.ups, encs[::-1]):
            x = up(x)
            x = x + enc_skip
            x = decoder(x)

        x = self.ending(x)
        x = x + inp

        return x[:, :, :H, :W]

    def check_image_size(self, x):
        _, _, h, w = x.size()
        mod_pad_h = (self.padder_size - h % self.padder_size) % self.padder_size
        mod_pad_w = (self.padder_size - w % self.padder_size) % self.padder_size
        x = F.pad(x, (0, mod_pad_w, 0, mod_pad_h))
        return x

class NAFNetLocal(Local_Base, NAFNet):
    def __init__(self, *args, train_size=(1, 3, 256, 256), fast_imp=False, **kwargs):
        Local_Base.__init__(self)
        NAFNet.__init__(self, *args, **kwargs)

        N, C, H, W = train_size
        base_size = (int(H * 1.5), int(W * 1.5))

        self.eval()
        with torch.no_grad():
            self.convert(base_size=base_size, train_size=train_size, fast_imp=fast_imp)


if __name__ == '__main__':
    import resource
    def using(point=""):
        # print(f'using .. {point}')
        usage = resource.getrusage(resource.RUSAGE_SELF)
        global Total, LastMem

        # if usage[2]/1024.0 - LastMem > 0.01:
        # print(point, usage[2]/1024.0)
        print(point, usage[2] / 1024.0)

        LastMem = usage[2] / 1024.0
        return usage[2] / 1024.0

    img_channel = 1
    img_ht = 960
    img_wd = 240
    width = 32
    
    enc_blks = [1, 1, 2, 8]
    middle_blk_num = 4
    dec_blks = [1, 1, 1, 1]
    
    print('enc blks', enc_blks, 'middle blk num', middle_blk_num, 'dec blks', dec_blks, 'width' , width)
    
    using('start . ')
    net = NAFNet(img_channel=img_channel, width=width, middle_blk_num=middle_blk_num, 
                      enc_blk_nums=enc_blks, dec_blk_nums=dec_blks)

    using('network .. ')

    # for n, p in net.named_parameters()
    #     print(n, p.shape)

    inp = torch.randn((4, img_channel, img_ht, img_wd))

    out = net(inp)
    final_mem = using('end .. ')
    # out.sum().backward()

    # out.sum().backward()

    # using('backward .. ')

    # exit(0)

    # keras like model summary
    summary(net, input_size=(img_channel, img_ht, img_wd), device='cpu')

    # inp_shape = (3, 512, 512)

    # from ptflops import get_model_complexity_info

    # macs, params = get_model_complexity_info(net, inp_shape, verbose=False, print_per_layer_stat=False)

    # params = float(params[:-3])
    # macs = float(macs[:-4])

    # print(macs, params)

    # print('total .. ', params * 8 + final_mem)





enc blks [1, 1, 2, 8] middle blk num 4 dec blks [1, 1, 1, 1] width 32
start .  11859.21875
network ..  11899.07421875
end ..  23004.7421875
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 960, 240]             320
       LayerNorm2d-2         [-1, 32, 960, 240]              64
            Conv2d-3         [-1, 64, 960, 240]           2,112
            Conv2d-4         [-1, 64, 960, 240]             640
        SimpleGate-5         [-1, 32, 960, 240]               0
 AdaptiveAvgPool2d-6             [-1, 32, 1, 1]               0
            Conv2d-7             [-1, 32, 1, 1]           1,056
            Conv2d-8         [-1, 32, 960, 240]           1,056
          Identity-9         [-1, 32, 960, 240]               0
      LayerNorm2d-10         [-1, 32, 960, 240]              64
           Conv2d-11         [-1, 64, 960, 240]           2,112
       SimpleGate-12       

In [13]:
# ------------------------------------------------------------------------
# Copyright (c) 2022 megvii-model. All Rights Reserved.
# ------------------------------------------------------------------------

'''

'''
import sys 
sys.path.append('/home/lllei/AI_localization/NAFNet')

import torch
import torch.nn as nn
import torch.nn.functional as F
from basicsr.models.archs.arch_util import LayerNorm2d
from basicsr.models.archs.local_arch import Local_Base
from torchsummary import summary


class SRCNNBlock(nn.Module):
    def __init__(self, c, c_expand=128, kernel_sizes=[(9, 9), (5, 5), (5, 5)], strides=[(1, 1), (1, 1), (1, 1)]):
        super().__init__()
        
        self.kernel_sizes = kernel_sizes
        self.strides = strides
        dw_channel = c * c_expand
        self.conv1 = nn.Conv2d(in_channels=c, out_channels=dw_channel, kernel_size=kernel_sizes[0], 
                                padding=0, stride=strides[0], groups=1, bias=True)
        self.conv2 = nn.Conv2d(in_channels=dw_channel, out_channels=dw_channel // 2, kernel_size=kernel_sizes[1], 
                                padding=0, stride=strides[1], groups=1, bias=True)
        self.conv3 = nn.Conv2d(in_channels=dw_channel // 2, out_channels=c, kernel_size=kernel_sizes[2], 
                                padding=0, stride=strides[2], groups=1, bias=True)

    def forward(self, inp):
        x = inp

        x = self.CircularPadding(x, 0)
        x = self.conv1(x)
        x = F.relu(x)

        x = self.CircularPadding(x, 1)
        x = self.conv2(x)
        x = F.relu(x)

        x = self.CircularPadding(x, 2)
        x = self.conv3(x)

        return x

    def CircularPadding(self, inp, iconv):
        _, _, H, W = inp.shape
        kht, kwd = self.kernel_sizes[iconv]
        sht, swd = self.strides[iconv]
        assert kwd%2 != 0 and kht%2 !=0 and (W-kwd)%swd==0 and (H-kht)%sht ==0, 'kernel_size should be odd, (dim-kernel_size) should be divisible by stride'

        pwd = int((W - 1 - (W - kwd) / swd) // 2)
        pht = int((H - 1 - (H - kht) / sht) // 2)
        
        x = F.pad(inp, (pwd, pwd, pht, pht), 'circular')

        return x


class SRCNNNet(nn.Module):

    def __init__(self, img_channel=1, c_expand=64, kernel_sizes=[(9, 9), (5, 5), (5, 5)], strides=[(1, 1), (1, 1), (1, 1)]):
        super().__init__()

        self.blk = SRCNNBlock(img_channel, c_expand, kernel_sizes, strides)

    def forward(self, inp):
        return self.blk(inp)


if __name__ == '__main__':
    import resource
    def using(point=""):
        # print(f'using .. {point}')
        usage = resource.getrusage(resource.RUSAGE_SELF)
        global Total, LastMem

        # if usage[2]/1024.0 - LastMem > 0.01:
        # print(point, usage[2]/1024.0)
        print(point, usage[2] / 1024.0)

        LastMem = usage[2] / 1024.0
        return usage[2] / 1024.0

    img_channel = 1
    img_ht = 960
    img_wd = 240
    strides = [(3, 1), (1, 1), (1, 1)]
    kernel_sizes = [(15, 9), (3, 3), (5, 5)]
    c_expand = 128

    print('kernel_sizes', kernel_sizes, 'strides', strides, 'c_expand', c_expand)
    
    using('start . ')
    net = SRCNNNet(img_channel=img_channel, c_expand=c_expand, kernel_sizes=kernel_sizes, strides=strides)

    using('network .. ')

    # for n, p in net.named_parameters()
    #     print(n, p.shape)

    inp = torch.randn((4, img_channel, img_ht, img_wd))

    out = net(inp)
    final_mem = using('end .. ')
    # out.sum().backward()

    # out.sum().backward()

    # using('backward .. ')

    # exit(0)

    # keras like model summary
    summary(net, input_size=(img_channel, img_ht, img_wd), device='cpu')

    # inp_shape = (3, 512, 512)

    # from ptflops import get_model_complexity_info

    # macs, params = get_model_complexity_info(net, inp_shape, verbose=False, print_per_layer_stat=False)

    # params = float(params[:-3])
    # macs = float(macs[:-4])

    # print(macs, params)

    # print('total .. ', params * 8 + final_mem)





kernel_sizes [(15, 9), (3, 3), (5, 5)] strides [(3, 1), (1, 1), (1, 1)] c_expand 128
start .  3532.35546875
network ..  3532.35546875
end ..  3532.35546875
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 128, 530, 240]          17,408
            Conv2d-2         [-1, 64, 530, 240]          73,792
            Conv2d-3          [-1, 1, 530, 240]           1,601
        SRCNNBlock-4          [-1, 1, 530, 240]               0
Total params: 92,801
Trainable params: 92,801
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.88
Forward/backward pass size (MB): 188.27
Params size (MB): 0.35
Estimated Total Size (MB): 189.50
----------------------------------------------------------------
